In [1]:
#%rm saved_models -r

In [2]:
"""%cd /content
!git clone -b before_change_to_model https://github.com/christianjensen2903/QuestionAnswering.git
%cd QuestionAnswering
!pip install -r requirements.txt
!git pull"""

/content
fatal: destination path 'QuestionAnswering' already exists and is not an empty directory.
/content/QuestionAnswering
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/christianjensen2903/QuestionAnswering
   7c667ad..4fa3dbf  before_change_to_model -> origin/before_change_to_model
Updating 7c667ad..4fa3dbf
Fast-forward
 models/GPT2Generator.py | 4 ++--
 1 file changed, 2 insertions(+), 2 deletions(-)


In [3]:
from models.Logistic.MultiGPT2Logistic import MultiGPT2Logistic
from models.Logistic.GPT2Logistic import GPT2Logistic
from languages.LanguageModel import LanguageModel
from DataExploration import DataExploration
from languages.Japanese import Japanese
from languages.English import English
from languages.Finnish import Finnish
from Preprocess import Preprocess
from models.Model import Model
from Pipeline import Pipeline
from typing import List
import transformers
import pandas as pd
import numpy as np
import datasets
import torch
import random

In [4]:
def enforce_reproducibility(seed=42):
    # Sets seed manually for both CPU and CUDA
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # For atomic operations there is currently 
    # no simple way to enforce determinism, as
    # the order of parallel operations is not known.
    # CUDNN
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # System based
    random.seed(seed)
    np.random.seed(seed)

enforce_reproducibility()

In [5]:
# Is used to minimize the clutter in the console
datasets.logging.set_verbosity_error()

# Define the languages to be used
languages: List[LanguageModel] = [
    Japanese(),
    Finnish(),
    English(),
]

# gpt2Generator = GPT2Generator()
torch.cuda.empty_cache()

In [6]:
all_data = {}

for language in languages:
    pipeline = Pipeline()

    # Get the preprocessed data and split it into training and validation data
    preprocessor = Preprocess(language.tokenize, language.clean)
    data = pipeline.get_data(language=language.name, preproccesor=preprocessor)
    train_data, validation_data = pipeline.split_data(data)

    all_data[language.name] = {
        "train_data": train_data,
        "validation_data": validation_data
    }


Loading data...

Loading data...

Loading data...


In [7]:
models = [
    GPT2Logistic()
]

In [8]:
question_beginning = {
    'english': ['Question: When', 'Question: What', 'Question: How'],
    'finnish': ['Question: Milloin', 'Question: Mikä', 'Question: Missä'],
    'japanese': ['Question: 日本', 'Question: 『', 'Question: アメリカ']
}

In [9]:
def test_gpt2(model, validation_data, language_name):
    for starting_word in question_beginning[language_name]:
        model.generate_text(starting_word)
    X_validation = model.extract_X(validation_data)
    y_validation = validation_data['is_answerable']
    print(model.get_perplexity(X_validation))

In [10]:
#all_data["japanese"]["train_data"].head(20)

In [11]:
transformers.logging.set_verbosity_info()

In [ ]:
for language in languages:
  print(f'\n ---- Language!: {language.name}')
  train_data = all_data[language.name]["train_data"]
  validation_data = all_data[language.name]["validation_data"]
  for model in models:
          model_name = model.__class__.__name__
          model.set_language(language.name)
          print(f'\n - Model: {model_name}')

          try:
              model.load()
          except:
              print('Extracting features...')
              X_train = model.extract_X(train_data)
              y_train = train_data['is_answerable']
              model = pipeline.train(
                  model,
                  X_train,
                  y_train
              )
              model.save()
          
          X_validation = model.extract_X(validation_data)
          y_validation = validation_data['is_answerable']
          pipeline.evaluate(
              model,
              X_validation,
              y_validation
          )
          test_gpt2(model.GPT2Generator, validation_data, language.name)

loading file spiece.model from cache at /root/.cache/huggingface/hub/models--rinna--japanese-gpt2-small/snapshots/d35a68cf1fea74b71708ce898b351471b5c698ce/spiece.model
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--rinna--japanese-gpt2-small/snapshots/d35a68cf1fea74b71708ce898b351471b5c698ce/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--rinna--japanese-gpt2-small/snapshots/d35a68cf1fea74b71708ce898b351471b5c698ce/tokenizer_config.json



 ---- Language!: japanese

 - Model: GPT2Logistic
Extracting features...


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--rinna--japanese-gpt2-small/snapshots/d35a68cf1fea74b71708ce898b351471b5c698ce/config.json
Model config GPT2Config {
  "_name_or_path": "rinna/japanese-gpt2-small",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 1,
  "embd_pdrop": 0.1,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": 3072,
  "n_layer": 12,
  "n_positions": 1024,
  "output_hidden_states": true,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
  

  0%|          | 0/7851 [00:00<?, ?ex/s]

The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 7851
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1473


Step,Training Loss


In [ ]:
validation_data = all_data["japanese"]["validation_data"].head(20)
validation_data["document_plaintext"].str.len()

In [ ]:
from transformers import T5Tokenizer, GPT2LMHeadModel

tokenizer = T5Tokenizer.from_pretrained(
    "rinna/japanese-gpt2-small",
    model_max_length=1024,
    truncation=True
  )
tokenizer.do_lower_case = True  # due to some bug of tokenizer config loading

model = GPT2LMHeadModel.from_pretrained("rinna/japanese-gpt2-small")

In [ ]:
#model_max_length

In [ ]:
data_point = all_data["japanese"]["validation_data"].iloc[11]
print(len(data_point["document_plaintext"]))
str_input = 'Question: ' + \
                data_point['question_text'] + '\nContext: ' + \
                data_point['document_plaintext']
str_input

In [ ]:
len(tokenizer.encode(str_input, truncation=True, padding=True, return_tensors='pt')[0])

In [ ]:
"""model(tokenizer.encode(str_input, truncation=True, return_tensors='pt')[0])"""